# Retrieval Augmented Question & Answering on Ynet data in Hebrew with Amazon Bedrock using LangChain & Amazon OpenSearch

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

### Context
Previously we saw that the model told us how to to change the tire, however we had to manually provide it with the relevant data and provide the contex ourselves. We explored the approach to leverage the model availabe under Bedrock and ask questions based on it's knowledge learned during training as well as providing manual context. While that approach works with short documents or single-ton applications, it fails to scale to enterprise level question answering where there could be large enterprise documents which cannot all be fit into the prompt sent to the model. 

### Pattern
We can improve upon this process by implementing an architecure called Retreival Augmented Generation (RAG). RAG retrieves data from outside the language model (non-parametric) and augments the prompts by adding the relevant retrieved data in context. 

In this notebook we explain how to approach the pattern of Question Answering to find and leverage the documents to provide answers to the user questions.

### Challenges
- How to manage large document(s) that exceed the token limit
- How to find the document(s) relevant to the question being asked

### Proposal
To the above challenges, this notebook proposes the following strategy
#### Prepare documents
![Embeddings](./images/Embeddings_lang.png)

Before being able to answer the questions, the documents must be processed and a stored in a document store index
- Scrape [Ynet](https://www.ynet.co.il/home/0,7340,L-8,00.html) news site.
- Process and split them into smaller chunks
- Create a numerical vector representation of each chunk using Amazon Bedrock Titan Embeddings model
- Create an index using the chunks and the corresponding embeddings
#### Ask question
![Question](./images/Chatbot_lang.png)

When the documents index is prepared, you are ready to ask the questions and relevant documents will be fetched based on the question being asked. Following steps will be executed.
- Create an embedding of the input question
- Compare the question embedding with the embeddings in the index
- Fetch the (top N) relevant document chunks
- Add those chunks as part of the context in the prompt
- Send the prompt to the model under Amazon Bedrock
- Get the contextual answer based on the documents retrieved

## Implementation
In order to follow the RAG approach this notebook is using the LangChain framework where it has integrations with different services and tools that allow efficient building of patterns such as RAG. We will be using the following tools:

- **LLM (Large Language Model)**: Anthropic Claude V2 available through Amazon Bedrock

  This model will be used to understand the document chunks and provide an answer in human friendly manner.
- **Embeddings Model**: Amazon Titan Embeddings available through Amazon Bedrock

  This model will be used to generate a numerical representation of the textual documents
- **Ynet scrapping function**: a function to scrapes articles from Ynet news site using beautifulsoup4 Python package 

- **Vector Store**: OpenSearch available through LangChain

  In this notebook we are using Amazon OpenSearch as a vector-store to store both the embeddings and the documents. 
- **Index**: VectorIndex

  The index helps to compare the input embedding and the document embeddings to find relevant document
- **Wrapper**: wraps index, vector store, embeddings model and the LLM to abstract away the logic from the user.

## Setup

Before running the rest of this notebook, you'll need to run the cells below to (ensure necessary libraries are installed and) connect to Bedrock.

In this notebook, we'll also need some extra dependencies:

- [OpenSearch Python Client](https://pypi.org/project/opensearch-py/), to store vector embeddings
- [beautifulsoup4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) to scrape the Ynet news site.

In [33]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for boto3>=1.28.57 from https://files.pythonhosted.org/packages/c7/dd/4fe47b2cec8731ec26d7410e659c4f0c4cd36baa835e2312cb0ec5383b07/boto3-1.28.65-py3-none-any.whl.metadata
  Obtaining dependency information for awscli>=1.29.57 from https://files.pythonhosted.org/packages/0e/d2/d47172d6159f07255cf07f28772d9d6536ba749432a2f566aa515c59094a/awscli-1.29.65-py3-none-any.whl.metadata
  Obtaining dependency information for botocore>=1.31.57 from https://files.pythonhosted.org/packages/63/c6/8e29a2b9dffa188d07c26d19ae578a26d8063834e4d844bf22c2a0028229/botocore-1.31.65-py3-none-any.whl.metadata
  Obtaining dependency information for s3transfer<0.8.0,>=0.7.0 from https://files.pythonhosted.org/packages/5a/4b/fec9ce18f8874a96c5061422625ba86c3ee1e6587ccd92ff9f5bf7bd91b2/s3transfer-0.7.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 89.0 MB/s eta 0:00:00
 

In [34]:
%pip install -U opensearch-py==2.3.1 langchain==0.0.309 beautifulsoup4==4.12.2 

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for urllib3<2,>=1.21.1 from https://files.pythonhosted.org/packages/b0/53/aa91e163dcfd1e5b82d8a890ecf13314e3e149c05270cc644581f77f17fd/urllib3-1.26.18-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 164.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.
sparkmagic 0.20.4 requires nest-asyncio==1.5.5, but you have nest-asyncio 1.5.6 which is incompatible.

[notice] A ne

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import json
import os
import sys
import boto3

boto3_bedrock = boto3.client(service_name='bedrock-runtime')

## Configure langchain

We begin with instantiating the LLM and the Embeddings model. Here we are using Anthropic Claude for text generation and Amazon Titan for text embedding.

Note: It is possible to choose other models available with Bedrock. You can replace the `model_id` as follows to change the model.

`llm = Bedrock(model_id="amazon.titan-tg1-large")`

Available model IDs include:

- `ai21.j2-ultra-v1`
- `ai21.j2-mid-v1`
- `amazon.titan-embed-text-v1`
- `amazon.titan-text-express-v1`
- `anthropic.claude-v1`
- `anthropic.claude-v2`
- `anthropic.claude-instant-v1`

In [4]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock
from langchain.load.dump import dumps
from langchain.docstore.document import Document


# - create the Anthropic Model
llm = Bedrock(
    model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={"max_tokens_to_sample": 4096}
)

# - create the Titan embeddings Model
embed_model_id = "amazon.titan-embed-text-v1"
bedrock_embeddings = BedrockEmbeddings(model_id=embed_model_id, client=boto3_bedrock)

## Data Preparation
Let's first define a function to scrape Ynet news site.

In [5]:
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests

def get_articles_from_ynet():
    url = "https://www.ynet.co.il"
    articles = []
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    sub_links = []
    for a in soup.find_all('a', href=True):
        if "article" in str(a['href']):
            sub_links.append(a['href'])

    sub_links = set(sub_links)
    # print(sub_links)

    print(f"Importing articles from: {url}. Found {len(sub_links)} sub links.")
    for link in tqdm(sub_links):
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        data = soup.find('script', attrs={'type': 'application/ld+json'})
        data = str(data)
        data = data.split("{", 1)[1]
        d = data.strip("</script>")
        d = "{" + d
        try:
            metaData = json.loads(d)
        except Exception as e:
            # print("Error loading article meta-data to json ", e)
            continue

        try:
            authors = metaData['author']['name'].split(',')  # to list
            keywords = metaData['keywords'].split(',')  # to list
            date = metaData['datePublished']

            article = {
                'title': metaData['headline'],
                'date_published': date,
                'authors': authors,
                'link': link,
                'keywords': keywords,
                'summary': metaData['description'],
                'text': metaData['articleBody'],
                'link': link
            }
        except Exception as e:
            # print("Error loading article data ", e)
            continue

        if article['text']:
            articles.append(article)

    return articles

Next, lets run the scraping function. It should take around 2 minutes to complete.  

In [6]:
%%time
articles = get_articles_from_ynet()

Importing articles from: https://www.ynet.co.il. Found 135 sub links.


100%|██████████| 135/135 [01:39<00:00,  1.36it/s]

CPU times: user 7.56 s, sys: 164 ms, total: 7.73 s
Wall time: 1min 39s


Let's inspect some of the fetched Ynet articles

In [7]:
for article in articles[:3]:
    print(f"title: {article['title']}")
    print(f"text: {article['text']}")
    print(f"link: {article['link']}")
    print(f"date_published: {article['date_published']}")
    print(f"summary: {article['summary']}")
    print(f"authors: {article['authors']}")
    print(f"keywords: {article['keywords']}")
    print("**************************")

title: ממחדשים: כדורי סושי מהאורז של אתמול וביסלי ביתי משאריות הפסטה 
text: כדורי סושי    זהו מתכון ניצול שאריות מושלם מכיוון שהוא מאפשר גיוון רב. אם נשארו לכם דגים משבת, אתם יכולים לפרק אותם ולהשתמש במקום הטונה ולנצל את הירקות שיש בבית למילוי. ערכים תזונתיים ליחידה: 72 קלוריות | 9.5 גרם פחמימות | 3 גרם חלבון | 2 גרם שומן |  12 יחידות החומרים: לאורז: 2 כוסות אורז מבושל מכל סוג  2 כפות מירין (או 1 כף חומץ + 1 כף מייפל) 1/2 כפית מלח למילוי: 1 קופסת טונה בשמן, מסוננת  1 מלפפון קצוץ דק 1 גזר קצוץ דק 1 אבוקדו חתוך לקוביות קטנות  1 גבעול בצל ירוק, קצוץ  לקישוט: שומשום שחור ולבן  אופן ההכנה: מניחים את האורז בקערה המתאימה למיקרו ומוסיפים 1/2 כוס מים רותחים. סוגרים ומחממים 4 דקות במיקרו. משאירים סגור 5 דקות נוספות ומוציאים.  מערבבים את האורז עם כל שאר המרכיבים.  מניחים שכבת אורז על ניילון נצמד או על דף שעווה. מסדרים מעל מעט מילוי בהרכב שאתם אוהבים במרכז שכבת האורז. סוגרים מכל הצדדים לכדור בעזרת הניילון הנצמד ומהדקים היטב.  משחררים את כדור האורז בעדינות, מפזרים מעט שומשום ומניחים במקרר. מכינים כ

Now we will build a list of Langchain [Document](https://docs.langchain.com/docs/components/schema/document) in order to use it later in the `RecursiveCharacterTextSplitter`.

In [8]:
docs = []
for article in articles:
    doc =  Document(
        page_content=article['text'], 
        metadata={
            "title": article['title'], 
            "link": article['link'], 
            "authors": article['authors'],
            "date_published": article['date_published'],
            "summary": article['summary'],
            "keywords": article['keywords']
        }
    )
    docs.append(doc)

Let's inspect the length of each of the articles we fetched from Ynet news site

In [9]:
len_array = []
for doc in docs:
    len_array.append(str(len(doc.page_content)))
print(', '.join(len_array))

5103, 3096, 1979, 1186, 3035, 2454, 5754, 1565, 7411, 14783, 3188, 2730, 1845, 6624, 2442, 11731, 2464, 4266, 1856, 1148, 3083, 3791, 2877, 3803, 6623, 3211, 1087, 1393, 1520, 4495, 2459, 5746, 7920, 6440, 4136, 4560, 3086, 3999, 2347, 2459, 1720, 9148, 1915, 7868, 2032, 4427, 2460, 3195, 1487, 2735, 3006, 2288, 1035, 634, 2883, 3015, 4614, 3600, 2408, 2423, 2050, 3185, 4460, 8331, 1983, 1895, 9566, 4668, 2693, 1742, 7785, 7284, 1708, 2650, 5452, 2872, 1227, 2029, 2298, 2232, 757, 2430, 3916, 3998, 9551, 4401, 2076, 4608, 985, 3281, 3286, 6724, 1306, 3438, 5745, 9755, 45919, 5260, 3755, 4782, 6315, 1747, 1786, 3376, 2650, 4741, 2659, 2981, 1963, 819, 2489, 35708, 15994, 361, 2655, 4663, 2181, 3958, 8704, 1588, 1242, 4071, 3361, 9910, 1345


The retrieved articles text should be large enough to contain enough information to answer a question; but small enough to fit into the LLM prompt. Also the embeddings model has a limit of the length of input tokens limited to 8k tokens, which roughly translates to ~32000 characters. For the sake of this use-case we are creating chunks of roughly 2000 characters with an overlap of 200 characters using [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html).

In [10]:
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=2000,
    chunk_overlap=200,
)
splits = text_splitter.split_documents(docs)

In [11]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents]) // len(
    documents
)
avg_char_count_pre = avg_doc_length(docs)
avg_char_count_post = avg_doc_length(splits)
print(f"Average length among {len(docs)} documents loaded is {avg_char_count_pre} characters.")
print(f"After the split we have {len(splits)} documents, which is more than the original {len(docs)}.")
print(
    f"Average length among {len(splits)} documents (after split) is {avg_char_count_post} characters."
)

Average length among 125 documents loaded is 4415 characters.
After the split we have 358 documents, which is more than the original 125.
Average length among 358 documents (after split) is 1668 characters.


In [12]:
sample_embedding = np.array(bedrock_embeddings.embed_query(docs[0].page_content))
modelId = bedrock_embeddings.model_id
print("Embedding model Id :", modelId)
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

Embedding model Id : amazon.titan-embed-text-v1
Sample embedding of a document chunk:  [-0.02064487  0.22352532 -0.1292298  ... -0.20077093  0.17227271
 -0.11000463]
Size of the embedding:  (1536,)


Following the similar pattern embeddings could be generated for the entire corpus and stored in a vector store.

Firt of all we have to create a vector store. In this workshop we will use ***Amazon OpenSerach serverless.***

Amazon OpenSearch Serverless is a serverless option in Amazon OpenSearch Service. As a developer, you can use OpenSearch Serverless to run petabyte-scale workloads without configuring, managing, and scaling OpenSearch clusters. You get the same interactive millisecond response times as OpenSearch Service with the simplicity of a serverless environment. Pay only for what you use by automatically scaling resources to provide the right amount of capacity for your application—without impacting data ingestion. 

In [13]:
identity = boto3.client('sts').get_caller_identity()['Arn']
identity

'arn:aws:sts::062083580489:assumed-role/AmazonSageMaker-ExecutionRole-20190829T190746/SageMaker'

The following step can take a minute to complete so be patient. 

In [14]:
import boto3
import time
vector_store_name = 'ynet-hebrew-rag-bedrock'
index_name = "ynet-hebrew-rag-bedrock-index"
encryption_policy_name = "ynet-hebrew-rag-bedrock-sp"
network_policy_name = "ynet-hebrew-rag-bedrock-np"
access_policy_name = 'ynet-hebrew-rag-bedrock-ap'

aoss_client = boto3.client('opensearchserverless')

security_policy = aoss_client.create_security_policy(
    name = encryption_policy_name,
    policy = json.dumps(
        {
            'Rules': [{'Resource': ['collection/' + vector_store_name],
            'ResourceType': 'collection'}],
            'AWSOwnedKey': True
        }),
    type = 'encryption'
)

network_policy = aoss_client.create_security_policy(
    name = network_policy_name,
    policy = json.dumps(
        [
            {'Rules': [{'Resource': ['collection/' + vector_store_name],
            'ResourceType': 'collection'}],
            'AllowFromPublic': True}
        ]),
    type = 'network'
)

collection = aoss_client.create_collection(name=vector_store_name,type='VECTORSEARCH')

while True:
    status = aoss_client.list_collections(collectionFilters={'name':vector_store_name})['collectionSummaries'][0]['status']
    if status in ('ACTIVE', 'FAILED'): break
    time.sleep(10)

access_policy = aoss_client.create_access_policy(
    name = access_policy_name,
    policy = json.dumps(
        [
            {
                'Rules': [
                    {
                        'Resource': ['collection/' + vector_store_name],
                        'Permission': [
                            'aoss:CreateCollectionItems',
                            'aoss:DeleteCollectionItems',
                            'aoss:UpdateCollectionItems',
                            'aoss:DescribeCollectionItems'],
                        'ResourceType': 'collection'
                    },
                    {
                        'Resource': ['index/' + vector_store_name + '/*'],
                        'Permission': [
                            'aoss:CreateIndex',
                            'aoss:DeleteIndex',
                            'aoss:UpdateIndex',
                            'aoss:DescribeIndex',
                            'aoss:ReadDocument',
                            'aoss:WriteDocument'],
                        'ResourceType': 'index'
                    }],
                'Principal': [identity],
                'Description': 'Easy data policy'}
        ]),
    type = 'data'
)

host = collection['createCollectionDetail']['id'] + '.' + os.environ.get("AWS_DEFAULT_REGION", None) + '.aoss.amazonaws.com:443'

Now we are ready to inject our documents into vector store. This can be easily done using [OpenSearch](https://python.langchain.com/docs/integrations/vectorstores/opensearch) implementation inside [LangChain](https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/faiss.html) which takes  input the embeddings model and the documents to create the entire vector store. Using the Index Wrapper we can abstract away most of the heavy lifting such as creating the prompt, getting embeddings of the query, sampling the relevant documents and calling the LLM. [VectorStoreIndexWrapper](https://python.langchain.com/en/latest/modules/indexes/getting_started.html#one-line-index-creation) helps us with that.

In [15]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from langchain.vectorstores import OpenSearchVectorSearch

service = 'aoss'
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, os.environ.get("AWS_DEFAULT_REGION", None), service)

docsearch = OpenSearchVectorSearch.from_documents(
    splits,
    bedrock_embeddings,
    opensearch_url=host,
    http_auth=auth,
    timeout = 100,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    index_name=index_name,
    engine="faiss",
)

## LangChain Vector Store and Querying

#### We can use the similarity search method to make a query and return the chunks of text without any LLM generating the response.

It takes a few seconds to make documents availible in index. **If you will get an empty output in a next cell, just wait a little bit and retry**. 

In [18]:
query = "מה מצב הכלכלה?"

results = docsearch.similarity_search(query, k=3)  # our search query  # return 3 most relevant docs
results

[Document(page_content='תבנו גשרים, אבל לוקח עשר שנים עד שמשלמים את הכסף. הכל איטי מדי. האמריקאים רוצים לעזור לישראל, גם כלכלית וגם צבאית, אבל גם אצלם זה עובר במנגנונים זמן רב. עם זאת, כדאי לזכור שבסופו של דבר כל הכספים האלו ישטפו אותנו. זה עניין של זמן". ״כדאי לזכור שבסופו של דבר כל הכספים האלו ישטפו אותנו. זה עניין של זמן״ אם למשברים יש דינמיקה דומה, למה לא ראינו עליות בשוק המניות? "על שוק המניות פועלים שני נתונים: הצפי לגבי רווחי החברות וגובה הריבית. אנחנו נמצאים עם שני סימני שאלה גדולים מאוד בנוגע לשניהם. לגבי הרווחיות של החברות, היום אף אחד לא קונה, לא מבלה ולא טס. כל משבר והזמן שלו. אחרי היציאה מהמשבר יש ריבאונד. בקורונה זה לקח שנה. בשלב הראשון יש שיתוק. בהרבה חברות העובדים לא נמצאים והן לא עובדות בתפוקה מלאה. אבל למדנו שזה זמני ושאחרי אירועים יש פיצוי". "הרי הממשלות לא יעילות ומי ייחלץ בסוף לעזרה? בנק ישראל", אומר כהנוביץ\'. "כרגע הנגיד עדיין זהיר, אבל יש כבר פעולות. בנק ישראל הודיע שהוא מוכר רזרבות מטבע חוץ ואמור לספק נזילות למוסדיים. הרי לא רחוק היום שנתחיל לשמוע טענות כמו \'ה

#### All of these are relevant results, telling us that the retrieval component of our systems is functioning. The next step is adding our LLM to generatively answer our question using the information provided in these retrieved contexts.

## Generative Question Answering

In generative question-answering (GQA), we pass our question to the Claude-2 but instruct it to base the answer on the information returned from our knowledge base. We can do this in LangChain easily using the RetrievalQA chain.

In [19]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

#### Let’s try this with our earlier query:

In [20]:
qa.run(query)

' לפי הטקסט, מצב הכלכלה בישראל צפוי להיפגע בחודשים הקרובים בעקבות מלחמת חרבות ברזל: \n\n- שיעור האבטלה, שהיה נמוך מאוד (3.4%) בחודש ספטמבר, צפוי לעלות בחדות בשל פיטורים המוניים בענפי התיירות, הבידור והמסעדנות.\n\n- השקל התחזק מול הדולר והאירו, והמדדים המובילים בבורסה ירדו במעט. \n\n- חברת הדירוג פיץ\' הציבה את דירוג האשראי של ישראל תחת "מעקב שלילי" בשל הסיכון הגאופוליטי.\n\n- מומחים צופים יציאת משקיעים זרים מהשקעות בישראל, מה שיתרום לפיחות נוסף של השקל.\n\n- בנק ישראל מנסה לייצב את המצב על ידי מכירת מט"ח, אך נראה שלא יוריד את הריבית בזמן הקרוב.\n\nכך שבסך הכל המצב הכלכלי בישראל צפוי להידרדר בחודשים הקרובים, אך קשה לאמוד בדיוק עד כמה.'

We’re still not entirely protected from convincing yet false hallucinations by the model, they can happen, and it’s unlikely that we can eliminate the problem completely. However, we can do more to improve our trust in the answers provided.

An effective way of doing this is by adding citations to the response, allowing a user to see where the information is coming from. We can do this by adding a parameter: `return_source_documents=True`.

In [21]:
qa_with_sources = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(search_kwargs={'k': 3}),return_source_documents=True)

In [22]:
qa_with_sources(query)

{'query': 'מה מצב הכלכלה?',
 'result': ' על פי הדיווחים נראה שמצב הכלכלה בישראל עומד להיפגע באופן משמעותי בעקבות מבצע "חרבות ברזל" בעזה, שהחל בתחילת אוקטובר 2022. \n\nשיעור האבטלה בישראל אמנם הגיע בספטמבר לשפל של 3.4%, אבל צפויה עלייה חדה באבטלה בשל המלחמה. כבר עתה ידוע על עשרות אלפי עובדים שנשלחו לחל"ת בענפי התיירות, הבידור והמסעדנות. \n\nהתחזית היא ששיעור האבטלה יעלה באופן משמעותי, אך טרם ידוע במדויק בכמה. לא ברור גם האם יאומץ שוב מנגנון החל"ת מימי הקורונה או לא.\n\nנראה אם כן שהמלחמה תפגע קשות בכלכלה הישראלית ותגרום לעלייה חדה באבטלה לאחר תקופה ארוכה של ירידה בשיעור המובטלים. עדיין מוקדם להעריך במדויק עד כמה, אך ברור שמדובר בפגיעה משמעותית.',
 'source_documents': [Document(page_content='תבנו גשרים, אבל לוקח עשר שנים עד שמשלמים את הכסף. הכל איטי מדי. האמריקאים רוצים לעזור לישראל, גם כלכלית וגם צבאית, אבל גם אצלם זה עובר במנגנונים זמן רב. עם זאת, כדאי לזכור שבסופו של דבר כל הכספים האלו ישטפו אותנו. זה עניין של זמן". ״כדאי לזכור שבסופו של דבר כל הכספים האלו ישטפו אותנו. זה עניין של זמן

#### Now we have answered the question being asked but also included the source of this information being used by the LLM.

#### We’ve learned how to ground Large Language Models with source knowledge by using a vector database as our knowledge base. Using this, we can encourage accuracy in our LLM’s responses, keep source knowledge up to date, and improve trust in our system by providing citations with every answer.

We can use this embedding of the query to then fetch relevant documents.
Now our query is represented as embeddings we can do a similarity search of our query against our data store providing us with the most relevant information.

### Customisable option
In the above scenario you explored the quick and easy way to get a context-aware answer to your question. Now let's have a look at a more customizable option with the helpf of [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html) where you can customize how the documents fetched should be added to prompt using `chain_type` parameter. Also, if you want to control how many relevant documents should be retrieved then change the `k` parameter in the cell below to see different outputs. In many scenarios you might want to know which were the source documents that the LLM used to generate the answer, you can get those documents in the output using `return_source_documents` which returns the documents that are added to the context of the LLM prompt. `RetrievalQA` also allows you to provide a custom [prompt template](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html) which can be specific to the model.

Note: In this example we are using Anthropic Claude as the LLM under Amazon Bedrock, this particular model performs best if the inputs are provided under `Human:` and the model is requested to generate an output after `Assistant:`. In the cell below you see an example of how to control the prompt such that the LLM stays grounded and doesn't answer outside the context.

In [25]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Human: אתה עיתונאי עוזר ומיידע. תסכם בשלוש-ארבע נקודות על פי המידע העדכני ביותר, מחודש אוקובר 2023. אם אתה לא יודע, תכתוב איני יודע.

{context}

Question: {question}
Assistant:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

qa_prompt = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT},
)
result = qa_prompt({"query": query})
print(result["result"])

 לסיכום המצב הכלכלי בישראל נכון לאוקטובר 2023:

- שיעור האבטלה בישראל ירד ל-3.4% בספטמבר, אך צפוי לעלות שוב בגלל מלחמת עזה. כבר ידוע על עשרות אלפי עובדים שנשלחו לחל"ת, בעיקר בתיירות ומסעדנות. 

- השקל המשיך להיחלש מול הדולר ונסחר סביב 4 ש"ח לדולר, עלייה של 0.4% ביום. המדדים בבורסה ירדו ב-0.1%.

- בנק ישראל רמז שלא בהכרח יוריד ריבית בקרוב, כדי לייצב את המטבע. 

- חברת הדירוג פיץ' הציבה את ישראל תחת "מעקב שלילי" בגלל הסיכון הגיאופוליטי, אך זה עדיין לא הורדת דירוג.

- המומחים צופים צעדי הרחבה מוניטרית ופיסקלית מהממשלה ובנק ישראל שיביאו להתאוששות כלכלית לאחר המלחמה.


### Manually search an article in OpenSearch Serverless

Sometimes you need more flexibility than Lanchain can provide. In those cases you can use low level `boto3` and `opensearch-py` APIs.

#### Get the mappings of our index

We 1st initialize an open search serverless client

In [26]:
open_search_serverless_client = OpenSearch(
        hosts=host,
        http_auth=auth,
        use_ssl=True,
        verify_certs=True,
        connection_class=RequestsHttpConnection,
        timeout=300
    )

In [27]:
index_data = open_search_serverless_client.indices.get_mapping(index=index_name)
print(dumps(index_data, pretty=True))

{
  "ynet-hebrew-rag-bedrock-index": {
    "mappings": {
      "properties": {
        "id": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword",
              "ignore_above": 256
            }
          }
        },
        "metadata": {
          "properties": {
            "authors": {
              "type": "text",
              "fields": {
                "keyword": {
                  "type": "keyword",
                  "ignore_above": 256
                }
              }
            },
            "date_published": {
              "type": "date"
            },
            "keywords": {
              "type": "text",
              "fields": {
                "keyword": {
                  "type": "keyword",
                  "ignore_above": 256
                }
              }
            },
            "link": {
              "type": "text",
              "fields": {
                "keyword": {
                  "type": "k

#### Query the index with KNN using embeddings from Bedrock Titan model

In [28]:
query

'מה מצב הכלכלה?'

Let's create embeddings for the query string using Titan embeddings model

In [29]:
response = boto3_bedrock.invoke_model(
                body=json.dumps({"inputText": query}),
                modelId=embed_model_id,
                accept='application/json',
                contentType='application/json'
            )
result = json.loads(response['body'].read())
embedded_search = result.get('embedding')
embedded_search[0:10]

[0.640625,
 0.484375,
 -0.32226562,
 -0.6171875,
 0.043701172,
 0.03857422,
 0.18554688,
 -0.0008773804,
 0.51171875,
 0.22070312]

In [30]:
vector_query = {
                "size": 5,
                "query": {"knn": {"vector_field": {"vector": embedded_search, "k": 2}}},
                "_source": False,
                "fields": ["text"]
            }

In [31]:
response = open_search_serverless_client.search(body=vector_query, index=index_name)
response["hits"]["hits"]

[{'_index': 'ynet-hebrew-rag-bedrock-index',
  '_id': '1%3A0%3AaocYQ4sB9u5z628DJQir',
  '_score': 0.0040803854,
  'fields': {'text': ['תבנו גשרים, אבל לוקח עשר שנים עד שמשלמים את הכסף. הכל איטי מדי. האמריקאים רוצים לעזור לישראל, גם כלכלית וגם צבאית, אבל גם אצלם זה עובר במנגנונים זמן רב. עם זאת, כדאי לזכור שבסופו של דבר כל הכספים האלו ישטפו אותנו. זה עניין של זמן". ״כדאי לזכור שבסופו של דבר כל הכספים האלו ישטפו אותנו. זה עניין של זמן״ אם למשברים יש דינמיקה דומה, למה לא ראינו עליות בשוק המניות? "על שוק המניות פועלים שני נתונים: הצפי לגבי רווחי החברות וגובה הריבית. אנחנו נמצאים עם שני סימני שאלה גדולים מאוד בנוגע לשניהם. לגבי הרווחיות של החברות, היום אף אחד לא קונה, לא מבלה ולא טס. כל משבר והזמן שלו. אחרי היציאה מהמשבר יש ריבאונד. בקורונה זה לקח שנה. בשלב הראשון יש שיתוק. בהרבה חברות העובדים לא נמצאים והן לא עובדות בתפוקה מלאה. אבל למדנו שזה זמני ושאחרי אירועים יש פיצוי". "הרי הממשלות לא יעילות ומי ייחלץ בסוף לעזרה? בנק ישראל", אומר כהנוביץ\'. "כרגע הנגיד עדיין זהיר, אבל יש כבר פעולות. בנ

### Clean up
You have reached the end of this workshop. Following cell will delete all created resources.


In [32]:
aoss_client.delete_collection(id=collection['createCollectionDetail']['id'])
aoss_client.delete_access_policy(name=access_policy_name, type='data')
aoss_client.delete_security_policy(name=encryption_policy_name, type='encryption')
aoss_client.delete_security_policy(name=network_policy_name, type='network')


{'ResponseMetadata': {'RequestId': 'c287002c-a8ee-4112-9e0e-f69847269843',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c287002c-a8ee-4112-9e0e-f69847269843',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'date': 'Wed, 18 Oct 2023 14:15:41 GMT'},
  'RetryAttempts': 0}}

## Conclusion
Congratulations on completing this moduel on retrieval augmented generation! This is an important technique that combines the power of large language models with the precision of retrieval methods. By augmenting generation with relevant retrieved examples, the responses we recieved become more coherent, consistent and grounded. You should feel proud of learning this innovative approach. I'm sure the knowledge you've gained will be very useful for building creative and engaging language generation systems. Well done!

In the above implementation of RAG based Question Answering we have explored the following concepts and how to implement them using Amazon Bedrock and it's LangChain integration.

- Loading documents and generating embeddings to create a vector store
- Retrieving documents to the question
- Preparing a prompt which goes as input to the LLM
- Present an answer in a human friendly manner
- keep source knowledge up to date, and improve trust in our system by providing citations with every answer.

### Take-aways
- Experiment with different Vector Stores
- Leverage various models available under Amazon Bedrock to see alternate outputs
- Explore options such as persistent storage of embeddings and document chunks
- Integration with enterprise data stores

# Thank You